#Gesture-Phase-Segmentation_load_dataset.ipynb
Loads the dataset from the [UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets/gesture+phase+segmentation) and returns train and test X/y numpy arrays.  This loader derived from TWRistAR version Feb'22.   Please follow the original dataset owners citation requests if you use this data in your work.

The basic flow is:
* Download and unzip the dataset if not already present
* Convert each recording *session* into Intermediate Representation 1 (IR1) format - a datetime indexed pandas dataframe with columns for each channel plus the label and subject number.
* Transform the IR1 into IR2 - a set of three numpy arrays containing sliding window samples
   * X = (samples, time steps per sample, channels)  
   * y =  (samples, label) # activity classification  
   * s =  (samples, subject) # subject number
* Clean and further transforms the IR2 arrays as needed - note the transforms that can be applied here are train vs test dependent.   For example, the IR2 arrays in the training set may be rebalanced, but those in the test set should not.
* Concatenate the processed IR2 arrays into the final returned train/validate/test arrays.

Set interactive to true to run the Jupyter Notebook version.  Note most of the calls are setup to test the functions, not process the entire dataset, to do that set interactive to false and run all so that main executes.   This notebook can be saved and run as a python file as well.


<a rel="license" href="http://creativecommons.org/licenses/by-sa/4.0/"><img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by-sa/4.0/88x31.png" /></a><br />This work is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by-sa/4.0/">Creative Commons Attribution-ShareAlike 4.0 International License</a>.

[Lee B. Hinkle](https://userweb.cs.txstate.edu/~lbh31/), Texas State University, [IMICS Lab](https://imics.wp.txstate.edu/)  
TODO:
* Some files (a1_raw) have time discontinuity.  Need to address when forming sliding windows.
* The one-hot encoding should be moved to a common function.


# Import Libraries and Common Load Dataset Code (from IMICS public repo)

In [1]:
import os
import shutil #https://docs.python.org/3/library/shutil.html
from shutil import unpack_archive # to unzip
import time
import pandas as pd
import numpy as np
from numpy import savetxt
from tabulate import tabulate # for verbose tables, showing data
from tensorflow.keras.utils import to_categorical # for one-hot encoding
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from time import gmtime, strftime, localtime #for displaying Linux UTC timestamps in hh:mm:ss
from datetime import datetime, date
import urllib.request # to get files from web w/o !wget
import matplotlib.pyplot as plt

In [2]:
def get_py_file(fname, url):
    """checks for local file, if none downloads from URL.    
    :return: nothing"""
    #fname = 'load_data_utils.py'
    #ffname = os.path.join(my_dir,fname)
    if (os.path.exists(fname)):
        print ("Local",fname, "found, skipping download")
    else:
        print("Downloading",fname, "from IMICS git repo")
        urllib.request.urlretrieve(url, filename=fname)

get_py_file(fname = 'load_data_utils.py', url = 'https://raw.githubusercontent.com/imics-lab/load_data_time_series/main/load_data_utils.py')
get_py_file(fname = 'load_data_transforms.py', url = 'https://raw.githubusercontent.com/imics-lab/load_data_time_series/main/load_data_transforms.py')

Local load_data_utils.py found, skipping download
Local load_data_transforms.py found, skipping download


In [3]:
import load_data_transforms as xform
import load_data_utils as utils

# Global Parameters

In [4]:
# environment and execution parameters
my_dir = '.' # replace with absolute path if desired
dataset_dir = os.path.join(my_dir,'gesture_phase_dataset') # Where dataset will be unzipped

interactive = True # for exploring data and functions interactively
verbose = True

In [5]:
# Gesture-Phase-Segmentation Dataset unique params
xform.time_steps = 30
xform.stride = 1
# the label map should contain all possible labels, it is used to convert from
# IR1 dataframe string labels of type categorical (saves a ton of memory) to 
# integers for the IR2 and beyond numpy ndarrays.
label_map_gps = {"label":     {"Rest": 0, "Preparation": 1, "Stroke": 2,
                                "Hold": 3, "Retraction": 4}}

In [6]:
interactive = False # don't run if interactive, automatically runs for .py version
verbose = False # to limit the called functions output

In [7]:
def get_gesture_phase_dataset():
    """checks for local zipfile, if none downloads from UCI repository
    after download will unzip the dataset into local directory.
    Assumes a global my_dir has been defined (default is my_dir = ".")
    :return: nothing"""
    zip_fname = 'gesture_phase_dataset.zip'
    zip_ffname = os.path.join(my_dir,zip_fname)
    gps_url = "https://archive.ics.uci.edu/ml/machine-learning-databases/00302/gesture_phase_dataset.zip"
    if (os.path.exists(zip_ffname)):
        if verbose:
            print ("Local",zip_ffname, "found, skipping download")
    else:
        print("Downloading Gesture-Phase-Segmentation dataset from UCI ML Repository")
        urllib.request.urlretrieve(gps_url, zip_fname)
    if (os.path.isdir(dataset_dir)):
        if verbose:
            print("Found existing directory:", dataset_dir, "skipping unzip")
    else:
        os.makedirs(dataset_dir)
        print("Unzipping Gesture Phase Segmentation file in", dataset_dir, "directory")
        if (os.path.exists(zip_ffname)):
            shutil.unpack_archive(zip_ffname,dataset_dir,'zip')
        else:
            print("Error: ", zip_ffname, " not found, exiting")
    return
if interactive:
    get_gesture_phase_dataset()

In [8]:
def get_gps_ir1_dict():
    """reads the Gesture-Phase-Segmentation raw .csv files in the global
    'dataset_dir'. This version uses dict to preserve original filenames.
    Returns: a dict containing IR1 dataframes."""
    fn_list = ['a1_raw.csv', 'a2_raw.csv', 'a3_raw.csv',
           'b1_raw.csv', 'b3_raw.csv', 'c1_raw.csv','c3_raw.csv']
    ir1_df_dict = dict() # an empty dictionary
    for item in fn_list:
        subject = item[0] # the first letter of the filename a,b,c
        story = item[1] # the number after subject letter, 1,2,3
        ffname = os.path.join(dataset_dir,item)
        # print(subject, story, ffname)
        df = pd.read_csv(ffname)
        # change to 32-bit, credit/ref https://stackoverflow.com/questions/69188132/how-to-convert-all-float64-columns-to-float32-in-pandas
        # Select columns with 'float64' dtype  
        float64_cols = list(df.select_dtypes(include='float64'))
        # The same code again calling the columns
        df[float64_cols] = df[float64_cols].astype('float32')
        # b1_raw has an instance of alternate spelling.  Change to match others.
        df['phase'] = df['phase'].replace({'Preparação':'Preparation'})
        # Seems better to explicitly type the other columns vs object.
        df['phase']=df['phase'].astype('category')

        df['sub'] = subject
        df['sub'] = [ ord(x) - 96 for x in df['sub']] # ord is unicode char
        df['sub']=df['sub'].astype('int8')
        df['story'] = story
        df['story']=df['story'].astype('int8')
        df.rename(columns={"phase": "label"}, inplace = True, errors="raise") # phase was GPS dataset specific

        # kinect sample rate is 15 or 30 fps, timestamp appears to
        # be a running counter not an actual UTC time.
        df['datetime'] = pd.to_datetime(df['timestamp'], unit='ms') 
        df.set_index('datetime',inplace=True)
        df = df.drop('timestamp', axis=1)
        ir1_df_dict[item.split('.')[0]]=df # key is root name of csv
    return ir1_df_dict
if interactive:
    df_dict = get_gps_ir1_dict()
    print(df_dict.keys())

# Option to save/pickle the IR1 dataframes.

In [9]:
if interactive: 
    save_dir = 'gesture_phase_raw_ir1'
    if (not os.path.isdir(save_dir)):
        os.mkdir(save_dir)

    for df_name, ir1_df in df_dict.items() :
        ffname = os.path.join(save_dir,df_name) + '.pkl'
        ir1_df.to_pickle(ffname)

In [10]:
if interactive: 
    my_df = df_dict['a1_raw']
    print(type(my_df))
    print(my_df.dtypes)
    display(my_df.head())
    my_df['lhx'].plot(figsize=(20, 10))

In [11]:
if interactive:
    # labels will plot if they have been converted to ints 
    my_df = xform.assign_ints_ir1_labels(df_dict['a1_raw'], label_mapping_dict = label_map_gps)
    my_df.plot(subplots=True, figsize=(20, 10)) # yay Pandas

In [12]:
# find timegaps and split so that the resulting IR1 dataframes
# represent a continuous recording session
# use resampe/mean/interpolate to fill small gaps and make
# larger ones NaN.   Then get a dataframe of just the NaN
# rows and use that index to split the dataframe
# NOTE:  This is dataset specific in that it only handles one big gap
# Interesting that the plots merge and don't show separately
if interactive:
    my_df = df_dict['a1_raw']
    max_gaps_to_fill = 30
    # good resample explanation https://datastud.dev/posts/time-series-resample
    rs_df = my_df.resample('33ms').mean().interpolate(limit = max_gaps_to_fill)
    display(rs_df['lhx'].plot(figsize=(20, 10))) # NaN will not be plotted by default
    nan_rows = rs_df[rs_df['lhx'].isnull()] # notnull also available
    if (len(nan_rows.index)==0):
        print("No gaps exceeding",max_gaps_to_fill,"found")
    else:
        print(len(nan_rows.index), "rows with gaps large than",max_gaps_to_fill,"found")
        gap_start, gap_end = nan_rows.index[0], nan_rows.index[-1]
        print("Start of gap", gap_start)
        print(" End of gap ", gap_end)
        df1 = rs_df.loc[rs_df.index[0]:gap_start]
        df1 = df1.iloc[:-max_gaps_to_fill] # drop the filled gap
        df2 = rs_df.loc[gap_end:rs_df.index[-1]]
        df2 = df2.iloc[1:] # drop the one NaN row due to slice
        display(df1['lhx'].plot(figsize=(20, 10)))
        display(df2['lhx'].plot(figsize=(20, 10)))



# Use Common Transforms to generate IR3 and final arrays

In [13]:
def get_ir3_from_dict(ir1_dict):
    """Processes a dictionary with key = IR1 source filename, item = IR1 df
    NOTE:  This version currently specific to Gesture dataset
    """
    ir3_X = np.zeros(shape=(1,xform.time_steps,19), dtype = 'float32')
    ir3_y = np.zeros(shape=(1,1),dtype='int8') # newer int method
    #ir3_y = np.full(shape=(1,1), fill_value='n/a',dtype='<U10') # unicode 10 char
    ir3_sub = np.zeros(shape=(1,1),dtype='int16') # one subject number per entry
    ir3_ss_times = np.zeros(shape=(1,2),dtype='datetime64') # start/stop times of sliding window
    for ir1_fname, ir1_df in ir1_dict.items():
        if verbose:
            print('Processing ', ir1_fname)
        ir1_df = xform.assign_ints_ir1_labels(ir1_df, label_mapping_dict = label_map_gps)
        ir2_X, ir2_y, ir2_sub, ir2_ss_time, channel_list = xform.get_ir2_from_ir1(ir1_df)
        ir2_X, ir2_y, ir2_sub, ir2_ss_time = xform.clean_ir2(ir2_X, ir2_y, ir2_sub, ir2_ss_time)
        ir2_X, ir2_y, ir2_sub, ir2_ss_time = xform.drop_label_ir2_ir3(ir2_X, ir2_y, ir2_sub, ir2_ss_time, 99)
        ir3_X = np.vstack([ir3_X, ir2_X])
        ir3_y = np.vstack([ir3_y, ir2_y])
        ir3_sub = np.vstack([ir3_sub, ir2_sub])
        ir3_ss_times = np.vstack([ir3_ss_times, ir2_ss_time])
    #delete first row placeholders
    X = np.delete(ir3_X, (0), axis=0) 
    y = np.delete(ir3_y, (0), axis=0) 
    sub = np.delete(ir3_sub, (0), axis=0)
    sub = np.delete(ir3_sub, (0), axis=0)
    ss_times = np.delete(ir3_ss_times, (0), axis=0)

    xys_info = 'Needs work!\n'
    # xys_info += '\n'.join([str(elem) for elem in zip_flist]) # conv list to string
    # xys_info += '\nTime steps =' + str(time_steps) + ', Step =' + str(stride) + ', no resample\n'
    # xys_info += 'Final Shapes\n'
    # xys_info += "X shape " + str(X.shape) + " dtype = " + str(X.dtype) + "\n"
    # xys_info += "y shape " + str(y.shape) + " dtype = " + str(y.dtype) + "\n"
    # xys_info += "sub shape " + str(sub.shape) + " dtype = " + str(sub.dtype) + "\n"
    xys_info += "IR1 Channel names:" + str(channel_list) + "\n"
    # # Get final counts for label ndarray - not quite as easy as pandas df
    # xys_info += "Final Label Counts\n"
    # unique, counts = np.unique(y, return_counts=True)
    # xys_info += str(np.asarray((unique, counts)).T)
    # xys_info += "\nSamples per Subject\n"
    # unique, counts = np.unique(sub, return_counts=True)
    # xys_info += str(np.asarray((unique, counts)).T)
    return X, y, sub, ss_times, xys_info
if interactive:
    X, y, sub, ss_times, xys_info = get_ir3_from_dict(df_dict)
    headers = ("array","shape", "object type", "data type")
    mydata = [("X:", X.shape, type(X), X.dtype),
            ("y:", y.shape ,type(y), y.dtype),
            ("sub:", sub.shape, type(sub), sub.dtype),
            ("ss_time:", ss_times.shape, type(ss_times), ss_times.dtype)]
    print(tabulate(mydata, headers=headers))
    unique, counts = np.unique(y, return_counts=True)
    print('Label Counts:\n',str(np.asarray((unique, counts)).T))
    print(label_map_gps)

In [14]:
def gesture_phase_segmentation_load_dataset(
    #verbose = False,
    #use_saved_xysub = False, # get X,y,sub from zip, True = faster to used saved ones
    incl_val_group = False, # split train into train and validate
    split_subj = dict
                (train_subj = [1,2,3],
                validation_subj = [],
                test_subj = []),
    #keep_channel_list = ['accel_ttl'],
    one_hot_encode = True, # make y into multi-column one-hot, one for each activity
    return_info_dict = False, # return dict of meta info along with ndarrays
    suppress_warn = False # special case for stratified warning
    ):
    """Downloads the Phase-Gesture-Segmentation dataset from UCI repository.
    Each csv file is converted into an IR1 dataframe and placed into a dictionary.
    Since this dataset is so small all subjects are placed into the train set
    and the validation set is a stratification of the train set.   The test set
    is empty by default (usually the test set is pulled out before model tuning)
    """
    log_info = "Generated by Gesture-Phase-Segmentation_load_dataset.ipynb\n"
    today = date.today()
    log_info += today.strftime("%B %d, %Y") + "\n"
    log_info += "sub dict = " + str(split_subj) + "\n"
    get_gesture_phase_dataset()
    df_dict = get_gps_ir1_dict()
    X, y, sub, ss_times, xys_info = get_ir3_from_dict(df_dict)
    #if (not use_saved_xysub):
    #   X, y, sub, xys_info = get_ir3()
    # Drop unwanted channels from X
    #log_info += "Keeping channels" + str(keep_channel_list) + "\n"
    #X = limit_channel_ir3(X, keep_channel_list = keep_channel_list)
    # write initial array info to log_info
    headers = ("Initial Array","shape", "object type", "data type")
    mydata = [("X", X.shape, type(X), X.dtype),
              ("y", y.shape, type(y), y.dtype),
              ("sub", sub.shape, type(sub), sub.dtype)]
    if (verbose):
        print(tabulate(mydata, headers=headers))
    log_info += tabulate(mydata, headers=headers) + "\n"

    if (one_hot_encode):
        # integer encode
        y_vector = np.ravel(y) #encoder won't take column vector
        le = LabelEncoder()
        integer_encoded = le.fit_transform(y_vector) #convert from string to int
        name_mapping = dict(zip(le.classes_, le.transform(le.classes_)))
        if (verbose):
            print("One-hot-encoding: category names -> int -> one-hot \n")
            print(name_mapping) # seems risky as interim step before one-hot
        log_info += "One Hot:" + str(name_mapping) +"\n\n"
        onehot_encoder = OneHotEncoder(sparse=False)
        integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
        onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
        y=onehot_encoded.astype('uint8')
        #return X,y
    # split by subject number pass in dictionary
    sub_num = np.ravel(sub[ : , 0] ) # convert shape to (1047,)
    # this code is different from typical due to limited subjects,
    # all not test subjects data is placed into train which is then 
    # split using stratification - validation group is not sub independent
    train_index = np.nonzero(np.isin(sub_num, split_subj['train_subj'] + 
                                        split_subj['validation_subj']))
    x_train = X[train_index]
    y_train = y[train_index]
    if (incl_val_group):
        if not suppress_warn:
            print("Warning: Due to limited subjects the validation group is a stratified")
            print("90/10 split of the training group.  It is not subject independent.")
        # split training into training + validate using stratify - note that the
        # validation set is not subject independent (hard to achieve with limited
        # subjects).   The test set however is subject independent and as a result
        # will have much lower accuracy.  Another option is to tag a few of the
        # activities for inclusion in validation.  See
        # https://github.com/imics-lab/Semi-Supervised-HAR-e4-Wristband
        # https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html
        x_train, x_validation, y_train, y_validation = train_test_split(x_train, y_train, test_size=0.10, random_state=42, stratify=y_train)

    test_index = np.nonzero(np.isin(sub_num, split_subj['test_subj']))
    x_test = X[test_index]
    y_test = y[test_index]

    headers = ("Returned Array","shape", "object type", "data type")
    mydata = [("x_train:", x_train.shape, type(x_train), x_train.dtype),
                    ("y_train:", y_train.shape ,type(y_train), y_train.dtype)]
    if (incl_val_group):
        mydata += [("x_validation:", x_validation.shape, type(x_validation), x_validation.dtype),
                        ("y_validation:", y_validation.shape ,type(y_validation), y_validation.dtype)]
    mydata += [("x_test:", x_test.shape, type(x_test), x_test.dtype),
                    ("y_test:", y_test.shape ,type(y_test), y_test.dtype)]
    if (verbose):
        print(tabulate(mydata, headers=headers))
    log_info += tabulate(mydata, headers=headers)
    if (incl_val_group):
        if (return_info_dict):
            return x_train, y_train, x_validation, y_validation, x_test, y_test, log_info
        else:
            return x_train, y_train, x_validation, y_validation, x_test, y_test
    else:
        if (return_info_dict):
            return x_train, y_train, x_test, y_test, log_info
        else:
            return x_train, y_train, x_test, y_test


# Main is setup to be a demo and bit of unit test.

In [15]:
if __name__ == "__main__":
    verbose = False
    print("Get Gesture Phase Segmentation using defaults - simple and easy!")
    x_train, y_train, x_test, y_test \
                             = gesture_phase_segmentation_load_dataset()
    headers = ("Array","shape", "data type")
    mydata = [("x_train:", x_train.shape, x_train.dtype),
            ("y_train:", y_train.shape, y_train.dtype),
            ("x_test:", x_test.shape, x_test.dtype),
            ("y_test:", y_test.shape, y_test.dtype)]
    print("\n",tabulate(mydata, headers=headers))
    print ('\n','-'*72)

    print("Get Gesture Phase Segmentation with validation and info file\n")
    x_train, y_train, x_valid, y_valid, x_test, y_test, log_info \
                             = gesture_phase_segmentation_load_dataset(
                                 incl_val_group = True,
                                 return_info_dict = True)

    headers = ("Array","shape", "data type")
    mydata = [("x_train:", x_train.shape, x_train.dtype),
            ("y_train:", y_train.shape, y_train.dtype),
            ("x_valid:", x_valid.shape, x_valid.dtype),
            ("y_valid:", y_valid.shape, y_valid.dtype),
            ("x_test:", x_test.shape, x_test.dtype),
            ("y_test:", y_test.shape, y_test.dtype)]
    print("\n",tabulate(mydata, headers=headers))
    print("\n----------- Contents of returned log_info ---------------")
    print(log_info)
    print("\n------------- End of returned log_info -----------------")

Get Gesture Phase Segmentation using defaults - simple and easy!

 Array     shape           data type
--------  --------------  -----------
x_train:  (2815, 30, 19)  float32
y_train:  (2815, 5)       uint8
x_test:   (0, 30, 19)     float32
y_test:   (0, 5)          uint8

 ------------------------------------------------------------------------
Get Gesture Phase Segmentation with validation and info file

90/10 split of the training group.  It is not subject independent.

 Array     shape           data type
--------  --------------  -----------
x_train:  (2533, 30, 19)  float32
y_train:  (2533, 5)       uint8
x_valid:  (282, 30, 19)   float32
y_valid:  (282, 5)        uint8
x_test:   (0, 30, 19)     float32
y_test:   (0, 5)          uint8

----------- Contents of returned log_info ---------------
Generated by Gesture-Phase-Segmentation_load_dataset.ipynb
February 26, 2023
sub dict = {'train_subj': [1, 2, 3], 'validation_subj': [], 'test_subj': []}
Initial Array    shape           obj

#Save arrays to drive
This is common code and untested - TWristAR is small so download and processing is fast.

For some of the larger datsets it is a big time benefit to store the arrays either before or after train/test split.  

In [16]:
if False: #change to true to save files interactively
    output_dir = '/content/drive/MyDrive/Processed_Datasets/TWristAR/all-sensors'
    if (os.path.isdir(output_dir)):
        #quick check for existing files, '.ipynb_checkpoints' file 
        #makes it more complicated to see if directory is empty
        if (not os.path.isfile(output_dir + '/X.npy')):
            summary = "TWristAR data\n"
            summary += "Saved to " + output_dir + "\n"
            summary += "Generated by TWristAR_load_data.ipynb"
            summary += " on " + time.strftime('%b-%d-%Y_%H%M', time.localtime())
            summary += "this version for fusion of learned representation work\n"
            summary += "contains data from all 4 e4 sensors"
            info_fname = output_dir +'/'+'README.txt'
            full_info = summary + "\n" + xys_info + "\n"
            print(full_info)

            with open(info_fname, "w") as file_object:
                file_object.write(full_info)

            if True:
                np.save(output_dir + '/'+'X.npy',X)
                np.save(output_dir + '/'+'y.npy',y)
                np.save(output_dir + '/'+'sub.npy',sub)
        else:
            print("Error "+output_dir+" contains X.npy, please delete files")
    else:
        print(output_dir + " not found, please create directory") 

In [17]:
if interactive:
    import matplotlib.pyplot as plt # for plotting

In [18]:
# Plot y - must convert to numeric first
def plot_activities():
    uniques, y_num = np.unique(y, return_inverse=True)
    print (uniques)
    plt.plot(y_num) 
    plt.show()
if (interactive):
    plot_activities()

In [19]:
def plot_subjects():
    uniques, s_num = np.unique(sub, return_inverse=True)
    print (uniques)
    plt.plot(s_num) 
    plt.show()
if (interactive):
    plot_subjects()